# Amazon Kindle Scraper

In [4]:
import ast
import requests
import json
import pandas as pd
import time

In [53]:
url = "https://amazon-kindle-scraper.p.rapidapi.com/products/B09WRPWY7S/reviews"

querystring = {"api_key":"bc09e263d60d1bbdfc2455c657c5e9bd"}

headers = {
    "X-RapidAPI-Key": "586fc518f3msh7a620ae644e3572p12100ejsn4956f8e89f67",
    "X-RapidAPI-Host": "amazon-kindle-scraper.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)
data = response.json()
print(response.text)

{"average_rating":4.7,"total_ratings":5616,"filtered_total_ratings":null,"total_reviews":null,"filtered_total_reviews":null,"5_star_ratings":4212,"5_star_percentage":0.75,"4_star_ratings":1011,"4_star_percentage":0.18,"3_star_ratings":337,"3_star_percentage":0.06,"2_star_ratings":56,"2_star_percentage":0.01,"1_star_ratings":56,"1_star_percentage":0.01,"product":{"url":"https://www.amazon.com/Riley-Thorn-Blast-Past-Score-ebook/dp/B09WRPWY7S/ref=cm_cr_arp_d_product_top?ie=UTF8","name":"Riley Thorn and the Blast from the Past","brand":"Lucy Score","price":null,"image":"https://images-na.ssl-images-amazon.com/images/I/41jWbOrdDWL.jpg","variation":{}},"top_positive_review":{"stars":4,"date":"Reviewed in the United States on May 18, 2022","verified_purchase":false,"manufacturer_replied":false,"username":"Amber Goleb","userUrl":"https://www.amazon.com/gp/profile/amzn1.account.AGAACFCKPGO54RHXNHA4PZNGPZIA/ref=cm_cr_arp_d_gw_lft?ie=UTF8","title":"More quirky fun","review":"Hello to my favorite 

In [14]:
#save to file

# with open("data.json",'w') as outfile:
#     json.dump(data,outfile)


#load file
f = open('data.json')
data = json.load(f)

data

{'average_rating': 4.7,
 'total_ratings': 5616,
 'filtered_total_ratings': None,
 'total_reviews': None,
 'filtered_total_reviews': None,
 '5_star_ratings': 4212,
 '5_star_percentage': 0.75,
 '4_star_ratings': 1011,
 '4_star_percentage': 0.18,
 '3_star_ratings': 337,
 '3_star_percentage': 0.06,
 '2_star_ratings': 56,
 '2_star_percentage': 0.01,
 '1_star_ratings': 56,
 '1_star_percentage': 0.01,
 'product': {'url': 'https://www.amazon.com/Riley-Thorn-Blast-Past-Score-ebook/dp/B09WRPWY7S/ref=cm_cr_arp_d_product_top?ie=UTF8',
  'name': 'Riley Thorn and the Blast from the Past',
  'brand': 'Lucy Score',
  'price': None,
  'image': 'https://images-na.ssl-images-amazon.com/images/I/41jWbOrdDWL.jpg',
  'variation': {}},
 'top_positive_review': {'stars': 4,
  'date': 'Reviewed in the United States on May 18, 2022',
  'verified_purchase': False,
  'manufacturer_replied': False,
  'username': 'Amber Goleb',
  'userUrl': 'https://www.amazon.com/gp/profile/amzn1.account.AGAACFCKPGO54RHXNHA4PZNGPZI

# get top reviews (both positive and negative)

In [15]:
review_json = data['reviews']
review_json.append(data['top_positive_review'])
review_json.append(data['top_critical_review'])

In [16]:
reviews = [x['review'] for x in review_json]

In [17]:
reviews

['I love Lucy!  And I love Riley Thorn & Nick Santiago.This is the third installment in her psychic detective series.  The mystery of what happened to Beth, Kellen Weber’s sister, is finally revealed.Just when you think the story is going one way, Lucy makes a sharp left turn and takes you on a hairpin curve wondering where you will finally end up.While Mrs. Penny is hot on the heels of the doody bandit and Nick is desperate to uncover the clues to lead him to Beth, Riley is working to regain her psychic energy.  Good thing she has Gabe, her spiritual guide, to help her.  Scary Grandmother’s, meddling family members and questionable choices make this another madcap adventure.',
 'What a fantastic book and series. The creative, stellar storyline and superb execution has NEVER been matched by other authors. I’m certain of it! All the hundreds of zany pieces to this story’s puzzle fit seamlessly together. The Audible companion is the icing on the cake. So entertaining. I listen to my book

# rank reviews based on 'total found helpful' info

In [18]:
total_found_helpful = []
for review in review_json:
    tot = review['total_found_helpful']
    if tot == None:
        tot = 0
    total_found_helpful.append(tot)

In [19]:
index_of_json = range(len(total_found_helpful))

In [21]:
import pandas as pd
review_df = pd.DataFrame(list(zip(index_of_json,reviews, total_found_helpful)),
              columns=['index','reviews','helpful'])

review_df = review_df.sort_values(by='helpful',ascending=False)
review_df.reset_index(drop=True,inplace=True)

In [22]:
review_df[:5] #top 5 - counting top critical and positive review too 

,index,reviews,helpful
0,10,"Hello to my favorite psychic, her band of quir...",15
1,9,Our favorite psychic and all-around trouble ma...,8
2,3,This book deserves a million stars!!! I read t...,2
3,11,I was so disappointed. I didn’t laugh one time...,2
4,2,Oh my goodness. I loved to first two Riley Tho...,1


# get top 5 reviews from ranked df (to save to MongoDB later)

In [31]:
review_df[:5]['reviews'].tolist()

["Hello to my favorite psychic, her band of quirky friends and family. I will always welcome more of these characters and their hijinks. This is a series that I have on my must read list for when new ones come out. I can't get enough of them.As always, Score entertains us with a fun and zany little mystery. While yes, it's a little out there. However, THAT IS THE POINT! These books are supposed to be fun. And they are. This one is no exception. I couldn't get enough of this book.We don't get as much of Riley and Nick's relationship. I mean, we do. Just not like we have before. This book concentrates more on the mystery. And I am okay with that. We have all the time in the world for more of their relationship in future books.I am sensing a spin-off. Am I the only one? That love-hate between a certain two characters has me wondering what we will get of them in the future. Ms. Score has me on the edge of my seat waiting to see.If you haven't picked up this book (or even the first two), I 

# Functions - to auto-update for all ISBNs

In [18]:
def getReviews(ISBN,return_dict):
    url = "https://amazon-kindle-scraper.p.rapidapi.com/products/"+ISBN+"/reviews"

    querystring = {"api_key":"bc09e263d60d1bbdfc2455c657c5e9bd"}

    headers = {
        "X-RapidAPI-Key": "78b650d129mshe8e75c4270e5d9cp1c535bjsn4566e6fce9d9",
        "X-RapidAPI-Host": "amazon-kindle-scraper.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    data = response.json()
    print(data)
    review_json = data['reviews']
    review_json.append(data['top_positive_review'])
    review_json.append(data['top_critical_review'])
    
    reviews = [x['review'] for x in review_json]
    
    
    total_found_helpful = []
    for review in review_json:
        tot = review['total_found_helpful']
        if tot == None:
            tot = 0
        total_found_helpful.append(tot)
        
    index_of_json = range(len(total_found_helpful))
    
    
    review_df = pd.DataFrame(list(zip(index_of_json,reviews, total_found_helpful)),
              columns=['index','reviews','helpful'])

    review_df = review_df.sort_values(by='helpful',ascending=False)
    review_df.reset_index(drop=True,inplace=True)
    
    return_dict['review'] = review_df[:5]['reviews'].tolist()  #top 5 - counting top critical and positive review too 
    return return_dict
    

In [16]:
def getGenre(ISBN,return_dict):
    
    url = "https://amazon-kindle-scraper.p.rapidapi.com/products/"+ISBN

    querystring = {"api_key":"bc09e263d60d1bbdfc2455c657c5e9bd"}

    headers = {
        "X-RapidAPI-Key": "78b650d129mshe8e75c4270e5d9cp1c535bjsn4566e6fce9d9",
        "X-RapidAPI-Host": "amazon-kindle-scraper.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    data = response.json()
    
    genre_list = []

    try: 
        text = data['product_information']['Best Sellers Rank']  #genre?

#         print(text)
        text = text.split('#')
        text = [x for x in text if x!='']
        for t in text:
            li = t.split('in',1)
            rank = li[0]
            genre = li[1]    
            parenthesis = re.search("[\(.*\)]",genre)
            if parenthesis:
                start = parenthesis.start()
                genre = genre[:start]

#             print(rank,genre)
            if genre.lower().strip() != 'books':
                genre_list.append(genre.strip())

    except KeyError:
        text = data['product_category']
        genre = text.split('›')[-1].strip()
        genre_list.append(genre)
    
    if len(genre_list) == 0:
        genre_list = ''
        
    return_dict['genre'] = genre_list
    
    return return_dict

In [45]:
subset_df = df[800:1000] #keep increasing first limit by 200 - due to API limitations
subset_df= subset_df.reset_index(drop=True)
subset_df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,url
0,0064407667,The Bad Beginning (A Series of Unfortunate Eve...,Lemony Snicket,1999,HarperCollins,http://images.amazon.com/images/P/0064407667.0...,http://images.amazon.com/images/P/0064407667.0...,http://images.amazon.com/images/P/0064407667.0...,https://www.goodreads.com/search?utf8=%E2%9C%9...
1,0007100221,TERROR FIRMA,Matthew Thomas,2001,Trafalgar Square,http://images.amazon.com/images/P/0007100221.0...,http://images.amazon.com/images/P/0007100221.0...,http://images.amazon.com/images/P/0007100221.0...,https://www.goodreads.com/search?utf8=%E2%9C%9...
2,2253148539,Le Loup-garou,Boris Vian,1999,LGF,http://images.amazon.com/images/P/2253148539.0...,http://images.amazon.com/images/P/2253148539.0...,http://images.amazon.com/images/P/2253148539.0...,https://www.goodreads.com/search?utf8=%E2%9C%9...
3,2253043974,Chronique d'une mort annoncÃ?Â©e,Gabriel Garcia Marquez,1987,LGF,http://images.amazon.com/images/P/2253043974.0...,http://images.amazon.com/images/P/2253043974.0...,http://images.amazon.com/images/P/2253043974.0...,https://www.goodreads.com/search?utf8=%E2%9C%9...
4,2020315491,Le neveu d'AmÃ?Â©rique,Luis SepÃ?Âºlveda,1998,Seuil,http://images.amazon.com/images/P/2020315491.0...,http://images.amazon.com/images/P/2020315491.0...,http://images.amazon.com/images/P/2020315491.0...,https://www.goodreads.com/search?utf8=%E2%9C%9...
...,...,...,...,...,...,...,...,...,...
195,0425166619,Toxin,Robin Cook,1999,Berkley Publishing Group,http://images.amazon.com/images/P/0425166619.0...,http://images.amazon.com/images/P/0425166619.0...,http://images.amazon.com/images/P/0425166619.0...,https://www.goodreads.com/search?utf8=%E2%9C%9...
196,061312975X,This Present Darkness,Frank E. Peretti,1999,Sagebrush Bound,http://images.amazon.com/images/P/061312975X.0...,http://images.amazon.com/images/P/061312975X.0...,http://images.amazon.com/images/P/061312975X.0...,https://www.goodreads.com/search?utf8=%E2%9C%9...
197,0765341972,The Mothman Prophecies,John A. Keel,2002,Tor Books,http://images.amazon.com/images/P/0765341972.0...,http://images.amazon.com/images/P/0765341972.0...,http://images.amazon.com/images/P/0765341972.0...,https://www.goodreads.com/search?utf8=%E2%9C%9...
198,0590514776,Meet the Stars of Buffy the Vampire Slayer,Stefanie Scott,1998,Scholastic,http://images.amazon.com/images/P/0590514776.0...,http://images.amazon.com/images/P/0590514776.0...,http://images.amazon.com/images/P/0590514776.0...,https://www.goodreads.com/search?utf8=%E2%9C%9...


# Main code - to get info for all the books

In [ ]:
count = 0
rejected_rows = []
json_obj = []

for row in range(len(subset_df)):
    
    ISBN = subset_df.loc[row,'ISBN']
#     print(ISBN)
    Book_Title = subset_df.loc[row,'Book-Title']
    Book_Author = subset_df.loc[row,'Book-Author']
    img_URL = subset_df.loc[row,'Image-URL-L']

    return_dict = {}
    return_dict['ISBN'] = ISBN
    return_dict['Book-Title'] = Book_Title
    return_dict['Book-Author'] = Book_Author
    return_dict['summary'] = ''
    return_dict['Image-URL-L'] = img_URL
    return_dict['reviews'] = return_dict['genre'] = ''

    return_dict = getReviews(ISBN,return_dict)
    return_dict = getGenre(ISBN,return_dict)

    #write to file 
    json_obj.append(return_dict)
    with open('data_final.json','w+')as outfile:
        json.dump(json_obj,outfile)

    print('row: ',str(count),'- done!')


    count+=1
    
    if count%10==0:
        time.sleep(120)
    

In [10]:
#rows that didn't run
rejected_rows

[]